<a href="https://colab.research.google.com/github/nbeaudoin/LangChain-Experimentation-Zone/blob/main/Summarization_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Tutorial source: https://www.youtube.com/watch?v=LNq_2s_H01Y&list=PL8motc6AQftk1Bs42EW45kwYbyJ4jOdiZ&index=8

In [ ]:
!pip -q install openai==0.27.0 langchain tiktoken python-dotenv

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
from dotenv import load_dotenv

# Load variables from file
load_dotenv('/content/drive/MyDrive/Projects/keys/secrets.json')

# Use variables
openai_api = os.getenv('OPENAI_API_KEY')

# Setting up Summarization Chain

In [ ]:
from langchain import OpenAI, PromptTemplate, LLMChain
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains.mapreduce import MapReduceChain
from langchain.prompts import PromptTemplate

llm = OpenAI(temperature=0)

In [ ]:
text_splitter = CharacterTextSplitter()

In [ ]:
# load the doc
with open('/content/drive/MyDrive/Projects/docs/alice_in_wonderland.txt') as f:
  alice_in_wonderland = f.read()
texts = text_splitter.split_text(alice_in_wonderland)

In [ ]:
len(texts)

4

In [ ]:
from langchain.docstore.document import Document

docs = [Document(page_content=t) for t in texts]

# Summarize with simple MapReduce

In [ ]:
from langchain.chains.summarize import load_summarize_chain
import textwrap

In [ ]:
chain = load_summarize_chain(llm,
                             chain_type="map_reduce")

output_summary = chain.run(docs)
wrapped_text = textwrap.fill(output_summary, width=100)
print(wrapped_text)

 Alice follows a white rabbit down a rabbit hole and falls into a deep well. She finds a tiny golden
key that fits a small door leading to a beautiful garden, but she is unable to fit through the door.
She drinks a bottle labeled "DRINK ME" and shrinks to ten inches tall, but is unable to reach the
key. She then eats a box labeled "EAT ME" and remains the same size, allowing her to enter the
garden.


In [ ]:
# for summarizing each part
chain.llm_chain.prompt.template

'Write a concise summary of the following:\n\n\n"{text}"\n\n\nCONCISE SUMMARY:'

In [ ]:
# for combining the parts
chain.combine_document_chain.llm_chain.prompt.template

'Write a concise summary of the following:\n\n\n"{text}"\n\n\nCONCISE SUMMARY:'

In [ ]:
chain = load_summarize_chain(llm,
                             chain_type="map_reduce",
                             verbose=True)

output_summary = chain.run(docs)
wrapped_text = textwrap.fill(output_summary,
                             width=100,
                             break_long_words=False,
                             replace_whitespace=False)

print(wrapped_text)



> Entering new MapReduceDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Write a concise summary of the following:


"CHAPTER I.
Down the Rabbit-Hole


Alice was beginning to get very tired of sitting by her sister on the
bank, and of having nothing to do: once or twice she had peeped into
the book her sister was reading, but it had no pictures or
conversations in it, “and what is the use of a book,” thought Alice
“without pictures or conversations?”

So she was considering in her own mind (as well as she could, for the
hot day made her feel very sleepy and stupid), whether the pleasure of
making a daisy-chain would be worth the trouble of getting up and
picking the daisies, when suddenly a White Rabbit with pink eyes ran
close by her.

There was nothing so _very_ remarkable in that; nor did Alice think it
so _very_ much out of the way to hear the Rabbit say to itself, “Oh
dear! Oh dear! I shall be late!” (when she thought it over afterwards,
it occ

# Summarizing with the "stuff" chain



*   Only makes one API call
*   Can't use with multiple pieces of data





In [ ]:
chain = load_summarize_chain(llm, chain_type="stuff")

In [ ]:
prompt_template = """ Write a concise bullet point summary of the following:

{text}

CONSCISE SUMMARY IN BULLET POINTS:"""

BULLET_POINT_PROMPT = PromptTemplate(template=prompt_template,
                                     input_variables=["text"])

BULLET_POINT_PROMPT

PromptTemplate(input_variables=['text'], template=' Write a concise bullet point summary of the following:\n\n{text}\n\nCONSCISE SUMMARY IN BULLET POINTS:')

In [ ]:
chain = load_summarize_chain(llm,
                             chain_type="stuff",
                             prompt=BULLET_POINT_PROMPT)

output_summary = chain.run(docs)

output_summary

'\n- Alice is bored and begins to consider making a daisy-chain when a White Rabbit with pink eyes runs by\n- Alice follows the Rabbit down a rabbit-hole and falls down a deep well\n- Alice finds herself in a long, low hall with doors all around, but they are all locked\n- Alice finds a little three-legged table with a tiny golden key and a bottle labelled “DRINK ME”\n- Alice drinks from the bottle and shrinks to the right size to fit through the little door into a beautiful garden\n- Alice finds a little glass box with a cake labelled “EAT ME” and eats it, but remains the same size'

In [ ]:
wrapped_text = textwrap.fill(output_summary,
                             width=100,
                             break_long_words = False,
                             replace_whitespace=False)

print(wrapped_text)


- Alice is bored and begins to consider making a daisy-chain when a White Rabbit with pink eyes
runs by
- Alice follows the Rabbit down a rabbit-hole and falls down a deep well
- Alice finds
herself in a long, low hall with doors all around, but they are all locked
- Alice finds a little
three-legged table with a tiny golden key and a bottle labelled “DRINK ME”
- Alice drinks from the
bottle and shrinks to the right size to fit through the little door into a beautiful garden
- Alice
finds a little glass box with a cake labelled “EAT ME” and eats it, but remains the same size


# Using "map reduce" with custom prompts

In [ ]:
chain = load_summarize_chain(llm,
                             chain_type="map_reduce",
                             map_prompt=BULLET_POINT_PROMPT,
                             combine_prompt=BULLET_POINT_PROMPT)

output_summary = chain.run(docs)
wrapped_text = textwrap.fill(output_summary,
                              width=100,
                              break_long_words=False,
                              replace_whitespace=False)

print(wrapped_text)


* Alice considers making a daisy-chain and follows a White Rabbit with pink eyes down a rabbit-hole
* She falls down a deep well and finds cupboards and book-shelves
* Alice finds a bottle with the
words "DRINK ME" printed on it and shrinks to 10 inches tall
* She then finds a small cake with the
words "EAT ME" printed on it and eats it, but remains the same size


In [ ]:
# with a custom prompt
prompt_template = """ Write a concise bullet point summary of the following:

{text}

CONSCISE SUMMARY IN BULLET POINTS:"""

PROMPT = PromptTemplate(template=prompt_template,
                                     input_variables=["text"])

# with intermediate steps
chain = load_summarize_chain(OpenAI(temperature=0),
                             chain_type="map_reduce",
                             return_intermediate_steps=True,
                             map_prompt=PROMPT,
                             combine_prompt=PROMPT)

output_summary = chain({"input_documents": docs}, return_only_outputs=True)
wrapped_text = textwrap.fill(output_summary["output_text"],
                                            width=100,
                                            break_long_words=False,
                                            replace_whitespace=False)

print(wrapped_text)


* Alice considers making a daisy-chain and follows a White Rabbit with pink eyes down a rabbit-hole
* She falls down a deep well and finds cupboards and book-shelves
* Alice finds a bottle with the
words "DRINK ME" printed on it and shrinks to 10 inches tall
* She then finds a small cake with the
words "EAT ME" printed on it and eats it, but remains the same size


In [ ]:
# Using intermediate steps
wrapped_text = textwrap.fill(output_summary["intermediate_steps"][2],
                                            width=100,
                                            break_long_words=False,
                                            replace_whitespace=False)

print(wrapped_text)



* Alice finds a bottle with the words "DRINK ME" printed on it
* She decides to look for a
"poison" label before drinking it
* The bottle has a mixed flavour of cherry-tart, custard, pine-
apple, roast turkey, toffee, and hot buttered toast
* After drinking it, Alice shrinks to 10 inches
tall
* She then finds a small cake with the words "EAT ME" printed on it
* Alice decides to eat the
cake, hoping it will make her grow larger or smaller so she can reach the key or fit through the
door to the garden


# With the "refine" CombineDocument Chain


*   Better responses
*   More API calls, cannot be paralleled since not independent



In [ ]:
chain = load_summarize_chain(llm, chain_type="refine")

output_summary = chain.run(docs)
wrapped_text = textwrap.fill(output_summary, width=100)
print(wrapped_text)

  Alice is bored and daydreaming when she sees a white rabbit with a pocket watch. She follows it
down a rabbit hole and falls into a deep well, passing shelves and cupboards as she falls and taking
a jar of marmalade. She wonders how far she has fallen and if she will ever reach the bottom. Alice
continues to fall and talks to herself, eventually coming upon a little three-legged table with a
tiny golden key on it. She uses the key to open a small door leading to a beautiful garden, but she
is unable to fit through the doorway. Alice begins to think that anything is possible and wonders if
she can somehow shrink herself to fit through the door. She finds a bottle with the words "DRINK ME"
printed on it and drinks it, shrinking to a height of ten inches. She finds a small cake with the
words "EAT ME" printed on it and eats it, growing to a height that allows her to fit through the
door and enter the garden. After eating a bit of the cake, Alice finds that she remains the same
size, so 